# Урок 4. Сверточные нейронные сети
# Домашнее задание

### <span class="burk">Задание 1.</span>
**Сделайте краткий обзор любой архитектуры для object detection.**

Проведите анализ: 
 - Чем отличается выбранная вами архитектура нейронной сети от других?
 - В чём плюсы и минусы данной архитектуры?
 - Какие могут возникнуть трудности при применении этой архитектуры на практике?


**SpineNet** — это backbone нейросетевая архитектура для задачи распознавания объекта. Разработкой модели занимались исследователи из Google Research. Модель обходит state-of-the-art подходы на задаче распознавания объектов на данных COCO. При этом SpineNet производит на 60% меньше вычислений и обходит ResNet-FPN на 6% по AP. Предложенную архитектуру также используют для классификации. SpineNet обходит state-of-the-art на 6% в точности на датасете iNaturalist.

 * Обычно backbone характеризует сеть с уменьшающимся масштабом в архитектуре “энкодер-декодер”, т.е. энкодер. 
 * Поскольку задача энкодера состоит в вычислении представлений признаков на основе вводных данных, backbone с уменьшающимся масштабом не сможет удерживать пространственную информацию. 
 * По мере углубления слоёв признаки будут становиться всё более абстрактными и менее локализованными, тем самым усложняя извлечение декодером точных необходимых признаков. 

С целью преодоления сложности получения и извлечения многомасштабных признаков для локализации была представлена модель с пермутируемыми масштабами и межмасштабными связями, предлагающая следующие улучшения:

1. Масштабы карт признаков в этой архитектуре получили возможность увеличения или уменьшения в любой момент времени посредством пермутации блоков, что противоположно прежнему шаблону, подразумевавшему только уменьшение. Это позволило сохранять пространственную информацию. 
2. Связям карт признаков разрешено пересекать масштабы признаков для выполнения слияния признаков из разных масштабов.

**_Summary:_**  

__Чем SpineNet отличается от предыдущих backbone-сетей?__  
В то время как для обнаружения присутствия признака может потребоваться высокое разрешение, для определения его локализации такая же точность не требуется.  

**SpineNet учится на разномасштабных изображениях за счет смешения слоев**

**_Плюсы архитектуры:_**

1. Использование SpineNet позволяет добиться хорошей точности при меньших затратах вычислительных ресурсов по сравнению с архитектурой ResNet
2. При использовании SpineNet во время энкодинга теряется меньше информации с изображения
3. Архитектура масштабируется: в [статье](https://arxiv.org/pdf/1912.05027.pdf) приводятся архитектуры SpineNet-49, SpineNet-96, SpineNet-143, SpineNet-190

**_Минусы архитектуры:_**  
1. SpineNet строится на основе архитектуры ResNet, следовательно все недостатки и ошибки ResNet присущи и SpineNet 
2. Размеры слоев подбирались с помощью нейронного поиска архитектур (NAS) что добавляет сложности при разработке модели


**_Вероятные сложности при практической реализации:_** 

- В связи с использованием в SpineNet слоев переменных масштабов потребуется дополнительные затраты времени для создания пермутированных слоев которые являются переходными между слоями с разными размерами изображения и количеством признаков.

#### Ссылки

1. Подробное объяснение архитектуры [SpineNet](https://www.youtube.com/watch?v=qFRfnIRMNlk) [EN]
2. Статья [SpineNet: Learning Scale-Permuted Backbone for Recognition and Localization](https://arxiv.org/pdf/1912.05027.pdf) [EN]
3. Разбор [SpineNet: нетрадиционная архитектура](https://nuancesprog.ru/p/10090/) [RU]

### <span class="burk">Задание 2$^*$.</span>
**Ссылка на репозиторий с полным кодом для обучения ssd нейросети:
https://github.com/sergeyveneckiy/ssd-tensorflow.** 

**Попробуйте улучшить точность её
работы и напишите отчёт (что вы пробовали изменить в её параметрах и как это
отражалось на процессе обучения нейронной сети)**

**Для MNIST:**  
 - Необходимо понизить количество входящих каналов до 2 (черный и белый)
 - Увеличить количество слоёв
 - Увеличить количество эпох
 - Тюнинг Dropout-ов  

**Для CIFAR100:**  
 - Увеличить количество выходных классов до 100
 - Возможно понадобится пересмотр архитектуры, так как в CIFAR100 классы объединены в суперклассы по 5. Таким образом сеть должна сначала отнести изображение к одному из 20 суперклассов, затем уже определять к какому из классов внутри суперкласса относится тестовое изображение.
 - Свертки и ядра возможно останутся такими же как в текущей модели, однако понадобится увеличение количества слоёв/эпох.
 - Тюнинг Dropout-ов

**Для IMAGENET:**  
 - Для анализа сета скорее всего потребуется ансамбль нейросетей, т.к. датасет очень большой.
 - Следует использовать подход сеть-внутри-сети
 - Учитывая что классов большое количество следует ограничить выборку.
 - Количество выходных классов нужно увеличить, также увеличить свертки/пулинги
 - Также понадобится увеличение количества слоёв/эпох

### <span class="girk">Выводы:</span>  
 1. Простая смена функции активации с ```relu``` на ```tanh``` + снижение ```Dropout``` в 2 раза позволила увеличить ```accuracy``` на тестовой выборке с 0,5 до 0,55
 2. Уменьшение размера партии c 128 до 64 (batch) уменьшает точность
 3. Замена функции активации в последнем слое активации с ```softmax``` на иную будет снижать точность. 
 4. Помимо конфигурирования слоев необходимо тюнить Dropout под каждую конфигурацию: уменьшение Convolution в первом слое с (3, 3) на (2, 2) и (1, 1) вместе с увеличением Dropout-ов с 0,06 до 0,1 привело к росту точности модели до 0,62.